## Forecasts for Product Demand
##### Make Accurate Forecasts for Thousands of Different Products

## 1. Data Description 
The dataset contains historical product demand for a manufacturing company with footprints globally. The company provides thousands of products within dozens of product categories. There are four central warehouses to ship products within the region it is responsible for. Since the products are manufactured in different locations all over the world, it normally takes more than one month to ship products via ocean to different central warehouses. If forecasts for each product in different central with reasonable accuracy for the monthly demand for month after next can be achieved, it would be beneficial to the company in multiple ways.

## 2. Objective: 
Is it possible to make forecasts for thousands of products (some of them are highly variable in terms of monthly demand) for the the month after next?

In [1]:
import pandas as pd
import numpy as np
import sys
import matplotlib

In [2]:
#Basic Data Analysis
df = pd.read_csv("../datasets/Historical Product Demand.csv")
print(f"Dataset has {df.shape[0]} rows and {df.shape[1]} columns")
df.head()

Dataset has 1048575 rows and 5 columns


,Product_Code,Warehouse,Product_Category,Date,Order_Demand
0,Product_0993,Whse_J,Category_028,2012/7/27,100
1,Product_0979,Whse_J,Category_028,2012/1/19,500
2,Product_0979,Whse_J,Category_028,2012/2/3,500
3,Product_0979,Whse_J,Category_028,2012/2/9,500
4,Product_0979,Whse_J,Category_028,2012/3/2,500


In [3]:
#all values are of type string or object need data type conversion
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 5 columns):
 #   Column            Non-Null Count    Dtype 
---  ------            --------------    ----- 
 0   Product_Code      1048575 non-null  object
 1   Warehouse         1048575 non-null  object
 2   Product_Category  1048575 non-null  object
 3   Date              1037336 non-null  object
 4   Order_Demand      1048575 non-null  object
dtypes: object(5)
memory usage: 40.0+ MB


In [4]:
#check rnage of values and distribution of each columns
df.describe().T

,count,unique,top,freq
Product_Code,1048575,2160,Product_1359,16936
Warehouse,1048575,4,Whse_J,764447
Product_Category,1048575,33,Category_019,481099
Date,1037336,1729,2013/9/27,2075
Order_Demand,1048575,3828,1000,112682


In [5]:
#There exisists some missing dates may require imputation
df.isnull().sum()

Product_Code            0
Warehouse               0
Product_Category        0
Date                11239
Order_Demand            0
dtype: int64

###### Product code can be used as an sku. Product category is a correlated column and Warehouse also doesn't act as a good feartue creates more biasness towards warehousr J. So These two columns can be dropped

In [6]:
#check different number warehouse and product categories
columns = ["Warehouse", "Product_Category"]
df[columns].value_counts()

Warehouse  Product_Category
Whse_J     Category_019        369078
           Category_001         93013
           Category_005         91215
Whse_A     Category_019         73390
Whse_J     Category_007         55192
                                ...  
Whse_S     Category_014            26
           Category_017            19
           Category_016            19
Whse_C     Category_008            19
Whse_A     Category_016            18
Length: 94, dtype: int64

In [7]:
#Format data types of colums
data = df.copy()

df["Date"] = pd.to_datetime(data["Date"])

data['Order_Demand'] = data['Order_Demand'].str.replace('(','-')
data['Order_Demand'] = data['Order_Demand'].str.replace(')','')
df["Order_Demand"] = pd.to_numeric(data["Order_Demand"])

d:\pluto7-work\demos or tasks\demandsensing\ds_env\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  
d:\pluto7-work\demos or tasks\demandsensing\ds_env\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  import sys


In [8]:
df.dtypes

Product_Code                object
Warehouse                   object
Product_Category            object
Date                datetime64[ns]
Order_Demand                 int64
dtype: object

In [28]:
#check if each product has only one category
print(f"Total Number of Products = {df['Product_Code'].nunique()}")
print(f"Total Number of Categories = {df['Product_Category'].nunique()}")

#iter
for product in df["Product_Code"].unique():
    temp = df[df["Product_Code"] == product]
    if len(temp["Product_Category"].unique()) > 1:
        print(f"Product {product} has multiple categories")
        
# as there is no multiple categories for product from above logic check value counts rows = number of products to cross verify
columns = ["Product_Code", "Product_Category"]
print(df[columns].value_counts())

Total Number of Products = 2160
Total Number of Categories = 33
Product_Code  Product_Category
Product_1359  Category_019        16936
Product_1295  Category_019        10575
Product_1378  Category_019         9770
Product_0620  Category_001         9428
Product_1286  Category_019         8888
                                  ...  
Product_1698  Category_021            2
Product_1703  Category_021            1
Product_0465  Category_012            1
Product_2099  Category_009            1
Product_0853  Category_021            1
Length: 2160, dtype: int64


In [78]:
#Drop Unwanted Columns and sort data by date
df = df.drop(columns=['Warehouse','Product_Category'])

# Sort data by date
df = df.sort_values('Date').reset_index().drop('index',axis=1)

In [79]:
print(f"The number of products is {len(df['Product_Code'].value_counts().index)}")
print(f"Dates min and Max ranges from {df['Date'].min()} to {df['Date'].max()}")


The number of products is 2160
Dates min and Max ranges from 2011-01-08 00:00:00 to 2017-01-09 00:00:00


In [80]:
# Extract negative values and aggregate by product
negative_orders_df = df.loc[df['Order_Demand'] < 0]
negative_pivot_table = pd.pivot_table(negative_orders_df, values='Order_Demand',index=['Product_Code'], aggfunc=np.sum).rename(columns={'Order_Demand':'Total_Neg'})

# Extract positve values and aggregate by product
positive_orders_df = df.loc[df['Order_Demand'] > 0]
positive_pivot_table = pd.pivot_table(positive_orders_df, values='Order_Demand',index=['Product_Code'], aggfunc=np.sum)

# Add a column with corresponding total positive value by product and calculate percentage, sort = desc
negative_pivot_table['Total_Pos'] = positive_pivot_table.loc[positive_pivot_table.index.isin(negative_pivot_table.index),]
negative_pivot_table['Percentage'] = abs(negative_pivot_table['Total_Neg'])*100/negative_pivot_table['Total_Pos']
negative_pivot_table = negative_pivot_table.sort_values('Percentage', ascending = False)

print("Percentage ranges from",negative_pivot_table['Percentage'].min(),"to",negative_pivot_table['Percentage'].max())

Percentage ranges from 0.0011126440874093195 to 82.183908045977


In [82]:
for product in df["Product_Code"].unique():
    pass
    #print(f"Order Demand of {product}")
    #print(df.loc[df["Product_Code"] == product].head(3))

In [87]:
df_null = df.loc[df['Date'].isnull() == True]
df = df.dropna(how='any',axis=0)
print("The number of products with null is",len(df_null['Product_Code'].value_counts().index))

[sum(df[i].isnull()) for i in df.columns]

# Covert float valued demand to absolute value
df['absolute_demand'] = abs(df['Order_Demand'])

# Remove all rows that have the same products, dates, and absolute values of demand
df_nodup = df.drop_duplicates(subset = ['Product_Code','Date','absolute_demand'], keep = False)

The number of products with null is 0


In [88]:
duplicates = df.iloc[~df.index.isin(df_nodup.index)]
duplicates = pd.pivot_table(duplicates, values='Order_Demand',index=['Product_Code','Date'],aggfunc=np.sum
                           ).reset_index()
duplicates = duplicates.loc[duplicates['Order_Demand']>0]

KeyError: 'Order_Demand'

In [85]:
df = df.drop_duplicates(subset = ['Product_Code','Date','absolute_demand'], keep = False)
df = df.drop('absolute_demand',axis=1)
df = df.loc[df['Order_Demand']>0]

# Add duplicates table back to data
df = pd.concat([df,duplicates], ignore_index = True)

# Check to see if any negative values remain
df.loc[df['Order_Demand'] <0] # no rows

NameError: name 'duplicates' is not defined